Importar librerias libreriales

In [1]:
import keras
import pandas as pd
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Reshape, Input, Concatenate
from keras.layers import Convolution1D as Conv1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm
import tensorflow as tf

print('Keras version: ',keras.__version__)

Keras version:  2.5.0


Importar dataset de Bruno

In [4]:
from google.colab import files
uploaded = files.upload()

Saving y_bsg5x7-cne.csv to y_bsg5x7-cne.csv
Saving x_bsg5x7-cne.csv to x_bsg5x7-cne.csv


In [89]:
df_X = pd.read_csv("x_bsg5x7-cne.csv", header=None)
df_y = pd.read_csv("y_bsg5x7-cne.csv", header=None)
y_origen = df_y[0]
y_destino = df_y[1]
df_X = pd.concat([df_X, y_origen], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_X, y_destino, test_size=0.38, random_state=69)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
X_train_2 = np.delete(X_train,range(35,51),1)
X_train_reshaped = np.expand_dims(X_train_2, axis=2)
print(X_train.shape,X_train_reshaped.shape, y_train.shape)

(5141, 51) (5141, 35, 1) (5141,)


In [90]:
X_train_reshaped.shape = (5141, 5, 7) # 5,y son las dimensiones de la bahía
X_train_reshaped[0]

array([[0.01587302, 0.01587302, 0.01587302, 0.33333333, 0.22222222,
        0.19047619, 0.19047619],
       [0.01587302, 0.01587302, 0.01587302, 0.01587302, 0.01587302,
        0.01587302, 0.06349206],
       [0.03174603, 0.23809524, 0.15873016, 0.06349206, 0.07936508,
        0.14285714, 0.25396825],
       [0.31746032, 0.20634921, 0.19047619, 0.12698413, 0.04761905,
        0.14285714, 0.26984127],
       [0.01587302, 0.17460317, 0.04761905, 0.28571429, 0.3015873 ,
        0.0952381 , 0.11111111]])

In [91]:
XX_train =X_train[0:,36:51]
XX_train.shape

(5141, 15)

In [71]:
# se hace un swap de la columna de origen con la columna 0 --> esto para facilitar el trabajo de la red
for i in range(len(X_train_reshaped)):
    origin = int(XX_train[i][-1])
    aux = X_train_reshaped[i][0]
    X_train_reshaped[i][0]=X_train_reshaped[i][origin]
    X_train_reshaped[i][origin]=aux
    if y_train[i]==0: y_train[i]=origin

In [92]:
y_train[:10]

array([0, 1, 1, 0, 2, 2, 1, 2, 4, 1])

In [93]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [94]:
from sklearn.preprocessing import OneHotEncoder

In [95]:
encoder = OneHotEncoder()
y_train_coded = encoder.fit_transform(y_train.reshape(-1,1)).toarray() 

In [96]:
y_train_coded[:10]

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]])

Modelado de la red

In [101]:
convmodel = Sequential()
convmodel.add(Conv1D(filters=30, kernel_size=3, activation='relu', input_shape=(7,1))) # los vectores tienen 7 elementos cada uno
#convmodel.add(Conv1D(filters=10, kernel_size=3, activation='relu'))
convmodel.add(Dropout(0.1))
convmodel.add(MaxPooling1D(pool_size=2))

In [102]:
#define two different inputs
input1 = Input((7, 1))
input2 = Input((7, 1))
input3 = Input((7, 1))
input4 = Input((7, 1))
input5 = Input((7, 1))
input6 = Input((15, ))
   
#use the model to get three different outputs (the same model)
out1 = convmodel(input1)
out2 = convmodel(input2)
out3 = convmodel(input3)
out4 = convmodel(input4)
out5 = convmodel(input5)

#concatenate the outputs and add the final part of your model: 
out = Concatenate()([out1,out2,out3,out4,out5])
out = Flatten()(out)
out = Concatenate()([out,input6])
out = Dense(500, activation='relu')(out)
out = Dense(100, activation='relu')(out)
out = Dense(50, activation='relu')(out)
#out = Dense(4, activation='softmax')(out)
out = Dense(5, activation='softmax')(out)   

#create the model taking 2 inputs with one output
model = Model([input1,input2,input3,input4,input5,input6],out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.save_weights('model.h5')
model.summary()


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 7, 1)]       0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           [(None, 7, 1)]       0                                            
__________________________________________________________________________________________________
input_45 (InputLayer)           [(None, 7, 1)]       0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           [(None, 7, 1)]       0                                            
____________________________________________________________________________________________

In [103]:
X_train1 = X_train_reshaped[0:,0]

print(X_train1.shape)

X_train2 = X_train_reshaped[0:,1]
X_train3 = X_train_reshaped[0:,2]
X_train4 = X_train_reshaped[0:,3]
X_train5 = X_train_reshaped[0:,4]

X_train1.shape=(5141, 7, 1)
X_train2.shape=(5141, 7, 1)
X_train3.shape=(5141, 7, 1)
X_train4.shape=(5141, 7, 1)
X_train5.shape=(5141, 7, 1)

model.load_weights('model.h5')
model.fit([X_train1,X_train2,X_train3,X_train4,X_train5,XX_train], y_train_coded, epochs=120, batch_size=64, verbose=True)

(5141, 7)
Epoch 1/120
81/81 [==============================] - 3s 8ms/step - loss: 1.5440 - accuracy: 0.2862
Epoch 2/120
81/81 [==============================] - 1s 8ms/step - loss: 1.2705 - accuracy: 0.5074
Epoch 3/120
81/81 [==============================] - 1s 9ms/step - loss: 1.1333 - accuracy: 0.5663
Epoch 4/120
81/81 [==============================] - 1s 9ms/step - loss: 1.0053 - accuracy: 0.6194
Epoch 5/120
81/81 [==============================] - 1s 8ms/step - loss: 0.9555 - accuracy: 0.6326
Epoch 6/120
81/81 [==============================] - 1s 8ms/step - loss: 0.8923 - accuracy: 0.6529
Epoch 7/120
81/81 [==============================] - 1s 9ms/step - loss: 0.8544 - accuracy: 0.6655
Epoch 8/120
81/81 [==============================] - 1s 8ms/step - loss: 0.7931 - accuracy: 0.7069
Epoch 9/120
81/81 [==============================] - 1s 8ms/step - loss: 0.7671 - accuracy: 0.7017
Epoch 10/120
81/81 [==============================] - 1s 9ms/step - loss: 0.7610 - accuracy: 0.7083

In [104]:
#
XX_test = X_test[0:,36:51]
print(XX_test.shape)
X_test_2 = np.delete(X_test,range(35,51),1)
X_test_reshaped = np.expand_dims(X_test_2, axis=2)
print(X_test_reshaped.shape)
X_test_reshaped.shape = (3152, 5, 7)

X_test1 = X_test_reshaped[0:,0]
X_test2 = X_test_reshaped[0:,1]
X_test3 = X_test_reshaped[0:,2]
X_test4 = X_test_reshaped[0:,3]
X_test5 = X_test_reshaped[0:,4]

X_test1.shape=(3152, 7, 1)
X_test2.shape=(3152, 7, 1)
X_test3.shape=(3152, 7, 1)
X_test4.shape=(3152, 7, 1)
X_test4.shape=(3152, 7, 1)


(3152, 15)
(3152, 35, 1)


In [105]:
m=model.predict([X_test1, X_test2, X_test3, X_test4, X_test5, XX_test])


In [106]:
#print(m)
m_final = np.argmax(m, axis=1)
print(m_final)
print(skm.accuracy_score(y_test, m_final))
print(skm.f1_score(y_test, m_final, average='macro'))
print(skm.mean_absolute_error(y_test, m_final))


[4 1 1 ... 0 4 4]
0.6821065989847716
0.6806636717071309
0.631979695431472
